In [46]:
import torch
import torch.nn as nn
from diffusers import DDPMPipeline, DDPMScheduler
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from copy import deepcopy

# --- 1. Classes Utilitaires (Hooks & Projection) ---

class FeatureHook:
    """Intercepte les activations d'une couche spécifique."""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
        self.features = None

    def hook_fn(self, module, input, output):
        self.features = output

    def close(self):
        self.hook.remove()

class ProjectionNet(nn.Module):
    """
    Réseau léger qui projette les features vers l'espace du watermark.
    Structure: GAP -> Linear -> ReLU -> Linear -> Sigmoid
    """
    def __init__(self, input_channels, watermark_len):
        super().__init__()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_channels, 256),
            nn.Sigmoid(),
            nn.Linear(256, watermark_len),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x: [Batch, Channels, H, W]
        # Global Average Pooling pour réduire la dimension spatiale
        if len(x.shape) == 4:
            x = x.mean(dim=[2, 3])
        return self.net(x)

# --- 2. Classe Principale DICTION ---

class DictionDDPM:
    def __init__(self, model_id, device="cuda"):
        self.device = device
        self.model_id = model_id

        # Chargement du modèle
        self.pipeline = DDPMPipeline.from_pretrained(model_id)
        self.unet = self.pipeline.unet.to(device)
        self.scheduler = self.pipeline.scheduler

        # Configuration par défaut
        self.config = {
            "layer_name": "mid_block.resnets.1.conv2", # Couche cible
            "watermark_len": 64,
            "trigger_size": 32, # Nombre d'images dans le trigger set
            "lr": 1e-4,
            "lambda_wat": 1.0,
            "epochs": 5
        }

    def _get_target_layer(self, model, layer_name):
        """Récupère le module PyTorch correspondant au nom."""
        for name, module in model.named_modules():
            if name == layer_name:
                return module
        raise ValueError(f"Couche {layer_name} introuvable.")

    def generate_trigger_set(self):
        """
        Génère un Trigger Set persistant (bruit + timesteps fixes).
        C'est ce qui servira d'entrée pour activer la marque.
        """
        # shape = (self.config["trigger_size"], 3, 32, 32) # CIFAR-10 shape
        shape=(self.config["trigger_size"],3,256,256) # CelebA-HQ shape

        # Bruit fixe
        trigger_noise = torch.randn(shape).to(self.device)

        # Timesteps fixes (choisis aléatoirement une fois)
        trigger_timesteps = torch.randint(
            0, self.scheduler.config.num_train_timesteps,
            (self.config["trigger_size"],),
            device=self.device
        ).long()

        return trigger_noise, trigger_timesteps

    def embed(self, dataloader):
        """
        Entraîne le modèle tatoué et le réseau de projection.
        Objectif :
          - Features Original -> Random Watermark
          - Features Tatoué -> Target Watermark
        """
        print(f"--- Démarrage Embedding DICTION ({self.config['layer_name']}) ---")

        # 1. Préparation des Modèles
        original_unet = deepcopy(self.unet)
        original_unet.eval() # Le modèle original est gelé (référence)
        for p in original_unet.parameters(): p.requires_grad = False

        watermarked_unet = self.unet
        watermarked_unet.train()

        # 2. Génération des Clés & Trigger Set
        trigger_noise, trigger_timesteps = self.generate_trigger_set()

        target_wm = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)
        random_wm = torch.randint(0, 2, (self.config["watermark_len"],)).float().to(self.device)

        # 3. Initialisation ProjNet (Dimension dynamique)
        # On fait un dummy pass pour avoir la taille des features
        dummy_layer = self._get_target_layer(watermarked_unet, self.config["layer_name"])
        dummy_hook = FeatureHook(dummy_layer)
        with torch.no_grad():
            _ = watermarked_unet(trigger_noise[:1], trigger_timesteps[:1]).sample
        input_channels = dummy_hook.features.shape[1]
        dummy_hook.close()

        proj_net = ProjectionNet(input_channels, self.config["watermark_len"]).to(self.device)
        proj_net.train()

        # 4. Optimiseur (Entraîne UNet + ProjNet)
        optimizer = torch.optim.AdamW(
            list(watermarked_unet.parameters()) + list(proj_net.parameters()),
            lr=self.config["lr"]
        )

        mse_loss = nn.MSELoss()
        bce_loss = nn.BCELoss()

        # --- BOUCLE D'ENTRAÎNEMENT ---
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(dataloader)
            for clean_images, _ in pbar:
                clean_images = clean_images.to(self.device)
                bs = clean_images.shape[0]

                # A. Tâche Principale (Fidelity) sur Batch aléatoire
                noise = torch.randn_like(clean_images).to(self.device)
                timesteps = torch.randint(0, self.scheduler.config.num_train_timesteps, (bs,), device=self.device).long()
                noisy_images = self.scheduler.add_noise(clean_images, noise, timesteps)

                optimizer.zero_grad()

                noise_pred = watermarked_unet(noisy_images, timesteps).sample
                l_main = mse_loss(noise_pred, noise)

                # B. Tâche DICTION (Sur Trigger Set)

                # 1. Extraire features ORIGINALES (Clean -> Random)
                orig_layer = self._get_target_layer(original_unet, self.config["layer_name"])
                hook_orig = FeatureHook(orig_layer)
                with torch.no_grad():
                    _ = original_unet(trigger_noise, trigger_timesteps).sample
                feat_orig = hook_orig.features
                hook_orig.close()

                # 2. Extraire features TATOUÉES (Watermarked -> Target)
                wat_layer = self._get_target_layer(watermarked_unet, self.config["layer_name"])
                hook_wat = FeatureHook(wat_layer)
                # Important: On garde le gradient ici !
                _ = watermarked_unet(trigger_noise, trigger_timesteps).sample
                feat_wat = hook_wat.features
                hook_wat.close()

                # 3. Projection & Loss
                # Le ProjNet doit apprendre à mapper Orig -> Random
                pred_orig = proj_net(feat_orig.detach()) # Detach car on ne touche pas à l'original
                l_proj_clean = bce_loss(pred_orig.mean(dim=0), random_wm)

                # Le ProjNet ET le UNet doivent apprendre Wat -> Target
                pred_wat = proj_net(feat_wat)
                l_proj_wat = bce_loss(pred_wat.mean(dim=0), target_wm)

                # Loss Totale
                l_wat = l_proj_clean + l_proj_wat
                l_total = l_main + self.config["lambda_wat"] * l_wat

                l_total.backward()
                optimizer.step()

                # Metrics
                ber = self._compute_ber(pred_wat.mean(dim=0), target_wm)
                pbar.set_description(f"L_Main: {l_main:.3f} | L_Wat: {l_wat:.3f} | BER: {ber:.2f}")

                # if ber == 0.0 and l_wat.item() < 0.05:
                #     print("✅ Convergence atteinte !")
                #     break
            # if ber == 0.0: break

        # Sauvegarde des éléments nécessaires pour l'extraction
        self.saved_keys = {
            "trigger_noise": trigger_noise,
            "trigger_timesteps": trigger_timesteps,
            "target_wm": target_wm,
            "proj_net": proj_net,
            "watermarked_unet": watermarked_unet,
            "original_unet": original_unet,
        }
        return watermarked_unet

    def extract(self, suspect_unet=None):
        """
        Extrait la marque d'un modèle suspect en utilisant les clés sauvegardées.
        """
        if suspect_unet is None:
            suspect_unet = self.saved_keys["watermarked_unet"]

        print("--- Extraction de la marque ---")
        suspect_unet.eval()
        proj_net = self.saved_keys["proj_net"]
        proj_net.eval()

        trigger_noise = self.saved_keys["trigger_noise"]
        trigger_timesteps = self.saved_keys["trigger_timesteps"]
        target_wm = self.saved_keys["target_wm"]

        # 1. Hook sur le modèle suspect
        target_layer = self._get_target_layer(suspect_unet, self.config["layer_name"])
        hook = FeatureHook(target_layer)

        # 2. Passage du Trigger Set
        with torch.no_grad():
            _ = suspect_unet(trigger_noise, trigger_timesteps).sample

        features = hook.features
        hook.close()

        # 3. Projection & BER
        wm_pred = proj_net(features).mean(dim=0)
        ber = self._compute_ber(wm_pred, target_wm)

        print(f"BER Extrait : {ber:.2f}")
        return ber, wm_pred

    @staticmethod
    def _compute_ber(pred, target):
        return ((pred > 0.5).float() != target).float().mean().item()



In [47]:
# --- EXEMPLE D'UTILISATION ---

# 1. Data Loader
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
# dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

from datasets import load_dataset
from torch.utils.data import DataLoader
import torch
from torchvision import transforms


import torch
import gc

gc.collect()
torch.cuda.empty_cache()

transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load from Hugging Face (no Google Drive issues)
print("Loading dataset...")
hf_dataset = load_dataset("nielsr/CelebA-faces", split="train")

class CelebAWrapper(torch.utils.data.Dataset):
    def __init__(self, hf_dataset, transform):
        self.dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image = self.dataset[idx]['image']
        if self.transform:
            image = self.transform(image)
        return image, 0

dataset = CelebAWrapper(hf_dataset, transform)
print("Dataset loaded!")

dataloader = DataLoader(dataset, batch_size=8, shuffle=True)
print("loader loaded!")





# 2. Instanciation & Embedding
# diction = DictionDDPM("google/ddpm-cifar10-32")
diction= DictionDDPM("google/ddpm-celebahq-256")

# Embed (Retourne le modèle tatoué)
watermarked_model = diction.embed(dataloader)

# 3. Extraction (Test immédiat)
ber, _ = diction.extract(watermarked_model)

Files already downloaded and verified


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/carbure/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/carbure/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


--- Démarrage Embedding DICTION (mid_block.resnets.1.conv2) ---


L_Main: 0.006 | L_Wat: 0.029 | BER: 0.00: 100%|██████████| 782/782 [01:38<00:00,  7.90it/s]
L_Main: 0.012 | L_Wat: 0.009 | BER: 0.00: 100%|██████████| 782/782 [01:36<00:00,  8.11it/s]
L_Main: 0.027 | L_Wat: 0.004 | BER: 0.00: 100%|██████████| 782/782 [01:40<00:00,  7.79it/s]
L_Main: 0.029 | L_Wat: 0.002 | BER: 0.00: 100%|██████████| 782/782 [01:39<00:00,  7.88it/s]
L_Main: 0.018 | L_Wat: 0.001 | BER: 0.00: 100%|██████████| 782/782 [01:38<00:00,  7.95it/s]

--- Extraction de la marque ---
BER Extrait : 0.00


In [57]:
import torch
import torch.nn.functional as F
from diffusers import UNet2DModel, DDPMScheduler
from torch.optim import AdamW
from tqdm import tqdm

def run_distillation_attack(diction_obj, dataloader, epochs=5, lr=1e-4):
    """
    Lance une distillation Black-Box (Output only) du Teacher tatoué vers un Student vierge.
    Monitore le BER (err_wat) à chaque époque.
    """
    device = diction_obj.device

    # --- 1. Récupération du Teacher (Gelé) ---
    teacher_unet = diction_obj.saved_keys["watermarked_unet"]
    # teacher_pipeline = DDPMPipeline.from_pretrained("google/ddpm-cifar10-32")
    # teacher_pipeline = DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
    # teacher_unet = teacher_pipeline.unet.to(device)

    teacher_unet.eval()
    for p in teacher_unet.parameters(): p.requires_grad = False

    # --- 2. Initialisation du Student (Vierge) ---
    print("\n--- Initialisation du Student ---")
    # On crée un modèle avec la même config mais des poids aléatoires
    # student_unet = UNet2DModel.from_config(teacher_unet.config).to(device)
    # student_pipeline = DDPMPipeline.from_pretrained("google/ddpm-cifar10-32")
    student_pipeline=DDPMPipeline.from_pretrained("google/ddpm-celebahq-256")
    student_unet = student_pipeline.unet.to(device)
    student_unet.train()

    # --- 3. Vérifications Avant Distillation (Sanity Checks) ---
    print("\n[Check 1] Vérification du Teacher (Doit être ~0.0)")
    ber_teacher, _ = diction_obj.extract(teacher_unet)
    if ber_teacher > 0.05:
        print(f"⚠️ ATTENTION : Le Teacher n'est pas bien tatoué (BER={ber_teacher:.2f})")
    else:
        print(f"✅ Teacher OK (BER={ber_teacher:.2f})")

    print("\n[Check 2] Vérification du Student (Doit être ~0.5 - Aléatoire)")
    ber_student_start, _ = diction_obj.extract(student_unet)
    print(f"ℹ️ Student avant distillation : BER={ber_student_start:.2f} (Normal pour un modèle vierge)")

    # --- 4. Configuration Distillation ---
    optimizer = AdamW(student_unet.parameters(), lr=lr)
    # Scheduler pour générer le bruit d'entraînement
    noise_scheduler = diction_obj.scheduler

    history = {"loss": [], "ber": []}

    print(f"\n--- Démarrage de la Distillation ({epochs} epochs) ---")
    a=0
    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
        running_loss = 0.0

        for clean_images, _ in pbar:
            clean_images = clean_images.to(device)
            bs = clean_images.shape[0]

            # A. Génération d'entrée (Bruit aléatoire, PAS le trigger set)
            noise = torch.randn_like(clean_images).to(device)
            timesteps = torch.randint(0, noise_scheduler.config.num_train_timesteps, (bs,), device=device).long()
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            # B. Teacher Prediction (Cible) - BLACK BOX (Juste la sortie)
            with torch.no_grad():
                target_pred = teacher_unet(noisy_images, timesteps).sample

            # C. Student Prediction
            student_pred = student_unet(noisy_images, timesteps).sample

            # D. Loss (MSE pure sur les sorties)
            loss = F.mse_loss(student_pred, target_pred)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(Loss_Distill=loss.item())

        # --- E. Vérification du Transfert de Marque (err_wat) ---
        # On utilise la méthode extract de diction sur le student actuel
        # Elle utilise le Trigger Set et le ProjNet du Teacher (les clés)
        print(f"\nCalcul du BER (err_wat) pour l'époque {epoch+1}...")
        current_ber, wat_ext = diction_obj.extract(student_unet)

        history["loss"].append(running_loss / len(dataloader))
        history["ber"].append(current_ber)

        print(f"👉 Fin Epoch {epoch+1} | Loss: {history['loss'][-1]:.4f} | BER Student: {current_ber:.2f} | ext_wat: {nn.BCELoss()(wat_ext, diction_obj.saved_keys['target_wm']).item():.4f}")

        # Condition de succès total (Si le student a parfaitement copié la marque)
        if current_ber==0.0 and a>=1:
            print("✅ Marque récupérée avec succès par distillation !")
            break
        elif current_ber==0.0 and a<1 :
            a+=1
        else:
            a=0
    return student_unet, history



In [58]:
# --- Lancement du test ---
# diction est l'objet créé dans l'étape précédente
# dataloader est votre chargeur CIFAR-10

student_distilled, stats = run_distillation_attack(diction, dataloader, epochs=1000)

Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/carbure/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/carbure/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.



--- Initialisation du Student ---


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/carbure/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/carbure/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


...


Calcul du BER (err_wat) pour l'époque 11...
--- Extraction de la marque ---
BER Extrait : 0.58
👉 Fin Epoch 11 | Loss: 0.0001 | BER Student: 0.58 | ext_wat: 4.2341


Epoch 12/100: 100%|██████████| 782/782 [01:29<00:00,  8.72it/s, Loss_Distill=5.15e-5] 



Calcul du BER (err_wat) pour l'époque 12...
--- Extraction de la marque ---
BER Extrait : 0.58
👉 Fin Epoch 12 | Loss: 0.0001 | BER Student: 0.58 | ext_wat: 4.2354


Epoch 13/100:  48%|████▊     | 375/782 [00:44<00:48,  8.43it/s, Loss_Distill=4.42e-5] 


KeyboardInterrupt: 